In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider, Dropdown, VBox, HBox, Button, interactive, Output
from IPython.display import display
from scipy import sparse
from skimage import data
from skimage.io import imread
from skimage.filters import gaussian

In [ ]:
patched_file_path = '/home/sysadmin/MS-2D-IMG/datasets/IBD_2D/top_512_patch_224x224_overlap_0x0_bin_size_0.01/ST000923-C8-pos/CD/top_512_patch_224x224_overlap_0x0_mz_200.0-1100.0_bin_size_0.01_0024_XAV_iHMP2_LIP_SM-6CAJC_CD.npz'

# image = np.load(patched_file_path)['patches']
# print(f"Data shape: {image.shape}")

In [ ]:
def create_interactive_viewer():
    cmaps = ['viridis', 'plasma', 'inferno', 'magma', 'cividis', 
             'gray', 'hot', 'cool', 'spring', 'summer', 'autumn', 'winter',
             'bone', 'copper', 'RdYlBu', 'Spectral']

    channel_slider = IntSlider(min=0, max=image.shape[0]-1, value=0, description='Channel:')
    vmin_slider = FloatSlider(min=float(image.min()), max=float(image.max()), 
                             value=np.percentile(image, 1), description='Min:')
    vmax_slider = FloatSlider(min=float(image.min()), max=float(image.max()),
                             value=np.percentile(image, 99), description='Max:')
    cmap_dropdown = Dropdown(options=cmaps, value='gray', description='Colormap:')
    zoom_slider = FloatSlider(min=0.1, max=2.0, value=1.0, description='Zoom:')
    output = Output()

    # auto range function
    def auto_range(_=None):
        current_channel = channel_slider.value
        vmin_slider.value = np.percentile(image[current_channel], 1)
        vmax_slider.value = np.percentile(image[current_channel], 99)

    # event handlers
    channel_slider.observe(auto_range, names='value')
    auto_range_btn = Button(description="Auto Range")
    auto_range_btn.on_click(auto_range)

    def update_plot(**kwargs):
        with output:
            output.clear_output(wait=True)
            plt.close('all')

            fig = plt.figure(figsize=(8*kwargs['zoom'], 8*kwargs['zoom']))
            plt.imshow(image[kwargs['channel']], 
                      cmap=kwargs['cmap'],
                      vmin=kwargs['vmin'],
                      vmax=kwargs['vmax'])
            plt.colorbar()
            plt.title(f'Channel {kwargs["channel"]}\nIntensity: {kwargs["vmin"]:.2f}-{kwargs["vmax"]:.2f}')
            plt.axis('off')
            plt.show()

    # create interactive controls
    controls = VBox([
        HBox([channel_slider, auto_range_btn]),
        HBox([vmin_slider, vmax_slider]),
        HBox([cmap_dropdown, zoom_slider])
    ])

    auto_range()
    interactive_plot = interactive(update_plot, 
                                  {'manual': False},
                                  channel=channel_slider,
                                  vmin=vmin_slider,
                                  vmax=vmax_slider,
                                  cmap=cmap_dropdown,
                                  zoom=zoom_slider)

    display(VBox([controls, output]))

In [ ]:
# create_interactive_viewer()

In [ ]:
def visualize_gaussian_smoothing(binned_file_path, smoothing_sigma):
    sparse_matrix = sparse.load_npz(binned_file_path)
    raw_image = sparse_matrix.toarray()
    # raw_image = raw_image / raw_image.max()
    
    if smoothing_sigma and smoothing_sigma > 0:
        smoothed_image = gaussian(raw_image, sigma=smoothing_sigma)
        plot_smoothed = True
    else:
        smoothed_image = raw_image
        plot_smoothed = False
        
    num_rows = 2 if plot_smoothed else 1
    fig, axes = plt.subplots(num_rows, 1, figsize=(4, 6 * num_rows), sharex=True, sharey=True)
    
    if num_rows == 1:
        ax_raw = axes
    else:
        ax_raw = axes[0]
        ax_smooth = axes[1]
        
    vmin = np.min(raw_image)
    vmax = np.max(raw_image)
    
    im_raw = ax_raw.imshow(raw_image, cmap='gray', aspect='auto', vmin=vmin, vmax=vmax)
    ax_raw.set_title('Raw Mass Spectrum Image')
    ax_raw.set_ylabel('m/z')
    fig.colorbar(im_raw, ax=ax_raw, label='Intensity')
    
    if plot_smoothed:
        im_smooth = ax_smooth.imshow(smoothed_image, cmap='gray', aspect='auto', vmin=vmin, vmax=vmax)
        ax_smooth.set_title(f'Smoothed Mass Spectrum Image (sigma={smoothing_sigma})')
        ax_smooth.set_ylabel('m/z')
        fig.colorbar(im_smooth, ax=ax_smooth, label='Intensity')
    else:
        ax_raw.set_xlabel('scan')
        
    plt.tight_layout()
    plt.show()

In [ ]:
binned_file_path = r'/home/sysadmin/MS-2D-IMG/datasets/IBD_2D_RAW/ST000923-C8-pos/mz_200.0-1100.0_bin_size_0.01/CD/mz_200.0-1100.0_bin_size_0.01_0024_XAV_iHMP2_LIP_SM-6CAJC_CD.npz'
smoothing_sigma = 0.5
visualize_gaussian_smoothing(binned_file_path, smoothing_sigma)